In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np
import sys
sys.path.append('modules')

import matplotlib.pyplot as plt

from sklearn import preprocessing
lbl = preprocessing.LabelEncoder()
import tensorflow as tf
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.grid_search import GridSearchCV
from scipy.sparse import csr_matrix, hstack
from scipy.stats import iqr
from sklearn.cross_validation import KFold, train_test_split
from keras.models import Sequential
from keras.layers.advanced_activations import PReLU
from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras.callbacks import EarlyStopping
from xgboost import XGBRegressor

%matplotlib inline

C:\Continuum\Anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\util\tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if d.decorator_argspec is not None), _inspect.getargspec(target))
C:\Continuum\Anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\util\tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if d.decorator_argspec is not None), _inspect.getargspec(target))
C:\Continuum\Anaconda3\envs\tf_gpu\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Continuum\Anaconda3\envs\tf_gpu\lib\site-packages\sklea

In [2]:
# XGBOOST model

train_xgb = pd.read_csv("input/train_fe.csv")
test_xgb = pd.read_csv('input/test_fe.csv')

final_test_xgb_id_test = test_xgb['id']

#Preparing test data
for col in test_xgb.columns:
    if test_xgb[col].dtype == 'object':
        
        lbl.fit(test_xgb[col].values)
        test_xgb[col] = lbl.transform(test_xgb[col].values)
   # else:
        #test_xgb[col] = test_xgb[col]
            
        
final_test_xgb_X = test_xgb.drop(['id', 'timestamp'], axis=1)

#Show the results 
print("Test data set has {} samples".format(final_test_xgb_X.shape))


# Data preprocessing and creating of test-train split of Train DATA
for col in train_xgb.columns:
    if train_xgb[col].dtype == 'object':
        lbl.fit(train_xgb[col].values)
        train_xgb[col] = lbl.transform(train_xgb[col].values)
        
# Training data test-train set preparation 
train_xgb_y = train_xgb['price_doc']
train_xgb_X = train_xgb.drop(['id', 'timestamp','price_doc'],axis=1)

train_xgb_y = np.array(train_xgb_y)
train_xgb_X = np.array(train_xgb_X)
# test-train splitting
xgb_train_X, xgb_test_X, xgb_train_y, xgb_test_y = train_test_split(train_xgb_X, train_xgb_y, test_size=0.3, random_state=0)
#Show the results
print("Train data set has {} features & {} labels".format(xgb_train_X.shape,xgb_train_y.shape ))   
print("Test data set has {} features & {} labels".format(xgb_test_X.shape,xgb_test_y.shape ))   

Test data set has (7662, 208) samples
Train data set has (21083, 208) features & (21083,) labels
Test data set has (9036, 208) features & (9036,) labels


In [3]:
# NN model

# importing train_mlp and test_mlp datasets
train_mlp=pd.read_csv("input/train_fe.csv")
test_mlp=pd.read_csv("input/test_fe.csv")

In [4]:
cols = train_mlp.columns
train_mlp.shape, test_mlp.shape

# marking train_mlp and test_mlp sets
n=train_mlp.shape[0]
train_mlp['data_set']=1
test_mlp['data_set']=0
test_mlp.price_doc=np.nan

final_test_mlp_id_test=test_mlp['id']

#logarithmic transformation of target did not better results
train_mlp.price_doc=(train_mlp.price_doc)
target_mlp=train_mlp.price_doc
train_mlp=train_mlp.append(test_mlp)
train_mlp.drop(['id'],axis=1,inplace=True)

# listing down binary features
binary=[]
for i in train_mlp:
    if train_mlp[i].dtypes=='object':
        #print(train_mlp[i].value_counts())
        if train_mlp[i].value_counts().shape[0]==2:
            binary.append(i)

# converting categorical varibales and handling missing data 
for i in binary:
    train_mlp[i]=pd.factorize(train_mlp[i])[0]
'''train_mlp.loc[train_mlp['ecology']=='no data','ecology_dat']=0
train_mlp.loc[train_mlp['ecology']!='no data','ecology_dat']=1
train_mlp.loc[train_mlp['ecology']=='no data','ecology']=np.nan
train_mlp.loc[train_mlp['ecology']=='poor','ecology']=1
train_mlp.loc[train_mlp['ecology']=='satisfactory','ecology']=2
train_mlp.loc[train_mlp['ecology']=='good','ecology']=3
train_mlp.loc[train_mlp['ecology']=='excellent','ecology']=4
train_mlp.ecology=pd.to_numeric(train_mlp.ecology)'''
train_mlp=pd.concat([train_mlp,pd.get_dummies(train_mlp.sub_area)],axis=1)

a=train_mlp.describe()
for i in a:
    train_mlp[i]=train_mlp[i].fillna((a.loc['min',i]-a.loc['max',i]*2))
    
train_mlp.drop(['timestamp','sub_area'],inplace=True,axis=1)

#from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
cols=train_mlp.columns.tolist()

train_mlp = pd.DataFrame(scaler.fit_transform(train_mlp), columns=cols)

# separating train_mlp and test_mlp after preprocessing

test_mlp=train_mlp[train_mlp['data_set']==0]
train_mlp=train_mlp[train_mlp['data_set']==1]

test_mlp.drop(['data_set','price_doc',],inplace=True,axis=1)
train_mlp.drop(['data_set','price_doc'],inplace=True,axis=1)

final_test_mlp_X = test_mlp
print("Test data set has {} samples".format(final_test_mlp_X.shape))

train_mlp = np.array(train_mlp)
target_mlp = np.array(target_mlp)

# test-train splitting
mlp_train_X, mlp_test_X, mlp_train_y, mlp_test_y = train_test_split(train_mlp, target_mlp, test_size=0.3, random_state=0)
#Show the results
print("Train data set has {} features & {} labels".format(mlp_train_X.shape,mlp_train_y.shape ))   
print("Test data set has {} features & {} labels".format(mlp_test_X.shape,mlp_test_y.shape )) 

Test data set has (7662, 353) samples
Train data set has (21083, 353) features & (21083,) labels
Test data set has (9036, 353) features & (9036,) labels


### Preparing the L0-models

In [5]:
reg_xgb = xgb.XGBRegressor(num_boost_round=100,
                           eta=0.1,
                           eval_metric='rmse',
                           objective='reg:linear',
                           seed=0,
                           gamma=0,
                           max_depth=8,
                           min_child_weight=6,
                           colsample_bytree=0.6,
                           subsample=0.9)


In [6]:
%%time
# XGBoost

folds = KFold(len(xgb_train_y), shuffle=False, n_folds=3)

for k, (train_index, test_index) in enumerate(folds):
    xtr = xgb_train_X[train_index]
    ytr = xgb_train_y[train_index]
    xte, yte = xgb_train_X[test_index], xgb_train_y[test_index]

    reg_xgb.fit(xtr, ytr)
    np.savetxt('ensemble/xgb_pred_fold_{}.txt'.format(k), (reg_xgb.predict(xte)))
    np.savetxt('ensemble/xgb_test_fold_{}.txt'.format(k), yte)

Wall time: 43.6 s


In [7]:
def mlp_model():
    model = Sequential()
    model.add(Dense(128, input_dim = mlp_train_X.shape[1],init='normal' ))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.1))

    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.1))
    model.add(Dense(1))
    # Compile Model
    model.compile(loss = 'mean_squared_error', optimizer = 'adam')
    return model

In [8]:
%%time
# MLP

folds = KFold(len(mlp_train_y), shuffle=False, n_folds=3)

for k, (train_index, test_index) in enumerate(folds):
    xtr = mlp_train_X [train_index]
    ytr = mlp_train_y[train_index]
    xte, yte = mlp_train_X[test_index], mlp_train_y[test_index]
    
    reg_mlp = mlp_model()
    fit = reg_mlp.fit(xtr, ytr, batch_size=32, epochs=100, verbose=0)
    pred = reg_mlp.predict(xte, batch_size=32)
    np.savetxt('ensemble/mlp_pred_fold_{}.txt'.format(k), pred)
    np.savetxt('ensemble/mlp_test_fold_{}.txt'.format(k), yte)

C:\Continuum\Anaconda3\envs\tf_gpu\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, kernel_initializer="normal", input_dim=353)`
  This is separate from the ipykernel package so we can avoid doing imports until


Wall time: 15min 16s


### Training on the whole dataset

<p> We train the same models on the whole training set (xgb_train_X, mlp_train_X and corresponding labels xgb_train_y, mlp_train_y) and generate predictions for the test set (xgb_test_X, mlp_test_X). <br>It is important to note that we do have labels for the test set, but we don't allow our L0-model see them instead it uses from the one of the 3 folds from the training set .
<p/>

In [9]:
%%time
# XGBoost on whole set and predicted using test set

reg_xgb.fit(xgb_train_X, xgb_train_y)
pred_xgb = reg_xgb.predict(xgb_test_X)
np.savetxt('ensemble/xgb_pred_test.txt'.format(k), pred_xgb)

Wall time: 21.6 s


In [10]:
%%time
#MLP on whole set and predicted using test set

reg_mlp = mlp_model()
fit = reg_mlp.fit(mlp_train_X, mlp_train_y, batch_size=32, epochs=100, verbose=0)
pred_mlp = reg_mlp.predict(mlp_test_X, batch_size=32)
np.savetxt('ensemble/mlp_pred_test.txt'.format(k), pred_mlp)

C:\Continuum\Anaconda3\envs\tf_gpu\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, kernel_initializer="normal", input_dim=353)`
  This is separate from the ipykernel package so we can avoid doing imports until


Wall time: 7min 50s


### L1-model training

<p>When the previous stage is completed, we have generated out-of-fold and test set predictions, which we can now use to train the stacker.<br/>Firstly, Loading out-of-fold predictions:<p/>

In [11]:
train_xgb1 = np.loadtxt('ensemble/xgb_pred_fold_0.txt')
train_xgb2 = np.loadtxt('ensemble/xgb_pred_fold_1.txt')
train_xgb3 = np.loadtxt('ensemble/xgb_pred_fold_2.txt')

train_mlp1 = np.loadtxt('ensemble/mlp_pred_fold_0.txt')
train_mlp2 = np.loadtxt('ensemble/mlp_pred_fold_1.txt')
train_mlp3 = np.loadtxt('ensemble/mlp_pred_fold_2.txt')

#### Sanity Check

<p>Checking if the stacked out-of-fold predictions are within the relavent limits.<p/>

In [12]:
# combing the out-of-fold predictions

train_xgb_folds = np.hstack((train_xgb1,train_xgb2,train_xgb3))
train_mlp_folds = np.hstack((train_mlp1,train_mlp2,train_mlp3))

In [13]:
# MSE of XGBoost combined predictions

mean_squared_error(xgb_train_y, train_xgb_folds)**0.5

2263499.0271548009

In [14]:
# MSE of MLP combined predictions

mean_squared_error(mlp_train_y, train_mlp_folds)**0.5

2525483.1708675493

<p>Both the above predictions for the combined out-of-fold predictions have an error within the expected range.  In order to get a baseline score for the stacker, let's calculate predictions with the entire training set.
<p/>

In [15]:
test_xgb_pred = np.loadtxt('ensemble/xgb_pred_test.txt')
test_mlp_pred = np.loadtxt('ensemble/mlp_pred_test.txt')

In [16]:
mse_xgb_test = mean_squared_error(xgb_test_y, test_xgb_pred)**0.5
mse_mlp_test = mean_squared_error(mlp_test_y, test_mlp_pred)**0.5
print ("Baseline MSE which we need to improve with stacking, XGB: {}; MLP: {}.".format(mse_xgb_test, mse_mlp_test))

Baseline MSE which we need to improve with stacking, XGB: 2238255.3947151075; MLP: 2530067.9384573186.


<p>The predictions from the whole training data sets a baseline score that the stacker should better
<p/>

### Training L1-model

This final step involves combining the L0-models (XGB and MLP). The selection of algorithm is very cruical, a simple regressor is preferable as it reduces the chances of overfitting. Hence, let's select linear regression model.

In [17]:
l1_train_x = np.vstack((train_xgb_folds, train_mlp_folds)).T
l1_test_x = np.vstack((test_xgb_pred, test_mlp_pred)).T
l1_train_y = mlp_train_y 
l1_test_y = mlp_test_y

In [18]:
# dimensions 
print ("Xtrain shape:", l1_train_x.shape)
print ("ytrain shape:", l1_train_y.shape)
print ("Xtest shape:", l1_test_x.shape)
print ("ytest shape:", l1_test_y.shape)

Xtrain shape: (21083, 2)
ytrain shape: (21083,)
Xtest shape: (9036, 2)
ytest shape: (9036,)


http://machinelearningmastery.com/implementing-stacking-scratch-python/
<p>We now fit a very basic linear regression and get the predictions for the final test set:</p>

In [19]:
reg = LinearRegression()

# Note that normalizing the data in case of linear models is very important
reg.fit((l1_train_x), (l1_train_y))
reg_pred = (reg.predict((l1_test_x)))

mse_stacker = mean_squared_error(l1_test_y, reg_pred)**0.5
mse_stacker

2232699.5288930475

In [20]:
print ("MAE for XGB:", mse_xgb_test)
print ("MAE for MLP:", mse_mlp_test)
print ("MAE for stacker:", mse_stacker)

MAE for XGB: 2238255.39472
MAE for MLP: 2530067.93846
MAE for stacker: 2232699.52889


In [21]:
reg.coef_

array([ 0.82578044,  0.20931594])

<p>The regression coefficient indicates the final prediction is a linear combination of XGB and MLP models and has following relationship:
 ### `Final_Prediction = 0.82*XGB_Pred + 0.20*MLP_Pred` 
<p>


The final ensemble model has CV error of 2233247.32635, that is best score achived compared to the individual models used in the ensemble.

## Final Predictions for Submission

In [22]:
final_test_mlp_X.shape, final_test_xgb_X.shape

((7662, 353), (7662, 208))

In [23]:
# xgb predictions
final_xgb_pred_y = reg_xgb.predict(np.array(final_test_xgb_X))
final_xgb_pred_y.shape

(7662,)

In [24]:
# mlp predictions

final_mlp_pred_y=reg_mlp.predict(final_test_mlp_X.as_matrix(), batch_size=32, verbose=0)
final_mlp_pred_y_rs = np.reshape(final_mlp_pred_y, final_mlp_pred_y.shape[0])
final_mlp_pred_y.shape, final_mlp_pred_y_rs.shape

((7662, 1), (7662,))

In [25]:
A = 0.82578044
B = 0.20931594

final_stacker_pred_y = A*final_xgb_pred_y +  B* final_mlp_pred_y_rs

In [26]:
subs=pd.DataFrame({'id':final_test_mlp_id_test.as_matrix(),'price_doc':final_stacker_pred_y})
subs.head()
subs.to_csv("output/final_stacker_pred_20170819.csv",index=False)

The final submission gave score of 0.32622, this the best score achieved compared to all the indidual models.